In [1]:
import ee
import sphobjinv as soi

In [2]:
inv = soi.Inventory()
inv

Inventory(project=None, version=None, source_type=<SourceTypes.Manual: 'manual'>)

In [3]:
inv.project = "earthengine-api"
inv.version = ee.__version__
inv

Inventory(project='earthengine-api', version='1.2.0', source_type=<SourceTypes.Manual: 'manual'>)

In [4]:
inv.objects.append(
    soi.DataObjStr(
        name=ee.__name__,
        domain="py",
        role=str(type(ee)),
        priority="1",
        uri="",
        dispname="-",
    )
)
inv

Inventory(project='earthengine-api', version='1.2.0', source_type=<SourceTypes.Manual: 'manual'>)

In [5]:
import json
from pathlib import Path

colab = "#colab-python"

# for every class in the list add itself to the inventory and all the submethods
class_list = json.loads(Path("inventory/ee_class_list.json").read_text())
for class_name in class_list:
    c = getattr(ee, class_name)
    inv.objects.append(
        soi.DataObjStr(
            name="ee." + class_name,
            domain="py",
            role="class",
            priority="1",
            uri="ee-" + class_name.lower(),
            dispname="-",
        )
    )

    # find all the methods in the object
    methods = [m for m in dir(c) if not m.startswith("_")]
    for method_name in methods:
        m = getattr(c, method_name)
        inv.objects.append(
            soi.DataObjStr(
                name="ee." + class_name + "." + method_name,
                domain="py",
                role="method",
                priority="1",
                uri="ee-" + class_name.lower() + "-" + method_name.lower().replace("_", "-"),
                dispname="-",
            )
        )

In [9]:
for f in ["Initialize", "Authenticate"]:
    inv.objects.append(
        soi.DataObjStr(
            name="ee." + f,
            domain="py",
            role="function",
            priority="1",
            uri="ee-" + f.lower().replace("_", "-"),
            dispname="-",
        )
    )

In [17]:
import inspect

methods = [
    m
    for m, o in inspect.getmembers(ee.data)
    if not (m.startswith("_") or inspect.isclass(o) or m[0].isupper())
]
for method_name in methods:
    inv.objects.append(
        soi.DataObjStr(
            name="ee.data." + method_name,
            domain="py",
            role="function",
            priority="1",
            uri="ee-data-" + method_name.lower().replace("_", "-"),
            dispname="-",
        )
    )

In [26]:
abstract_classes = [
    cls for name, cls in inspect.getmembers(ee.batch.Export) if not name.startswith("_")
]
for abstract_class in abstract_classes:
    methods = [
        meth for name, meth in inspect.getmembers(abstract_class) if not name.startswith("_")
    ]
    for method in methods:
        inv.objects.append(
            soi.DataObjStr(
                name="ee.batch.Export." + abstract_class.__name__ + "." + method.__name__,
                domain="py",
                role="method",
                priority="1",
                uri="ee-batch-export-"
                + abstract_class.__name__
                + "-"
                + method.__name__.lower().replace("_", "-"),
                dispname="-",
            )
        )

[<function ee.batch.Export.classifier.toAsset(classifier, description='myExportClassifierTask', assetId=None, overwrite=False, priority=None, **kwargs)>]

In [20]:
inspect.getmembers(ee.batch)

[('Any', typing.Any),
 ('Dict', typing.Dict),
 ('Export', ee.batch.Export),
 ('FORMAT_PREFIX_MAP', {'GEOTIFF': 'tiff', 'TFRECORD': 'tfrecord'}),
 ('IMAGE_FORMAT_FIELD', 'fileFormat'),
 ('IMAGE_FORMAT_OPTIONS_FIELD', 'formatOptions'),
 ('List', typing.List),
 ('NON_FILE_DESTINATIONS',
  frozenset({<ExportDestination.ASSET: 'ASSET'>,
             <ExportDestination.BIGQUERY: 'BIGQUERY'>,
             <ExportDestination.FEATURE_VIEW: 'FEATURE_VIEW'>})),
 ('Optional', typing.Optional),
 ('Sequence', typing.Sequence),
 ('Task', ee.batch.Task),
 ('Union', typing.Union),
 ('__builtins__',
  {'__name__': 'builtins',
   '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built

In [6]:
text = inv.data_file(contract=True)
ztext = soi.compress(text)
soi.writebytes("earthengine-api.inv", ztext)